<a href="https://colab.research.google.com/github/Gabriele-Raffaele/concept_gridlock/blob/master/DCG_convert_chunk1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Raw Readers

In [ ]:
!git clone https://github.com/Gabriele-Raffaele/openpilot.git

fatal: destination path 'openpilot' already exists and is not an empty directory.


In [ ]:
%pwd


'/content'

In [ ]:
!pip install pycapnp
!pip install smbus2
!pip install lru-dict

In [ ]:
from __future__ import print_function
import os
import numpy as np
import sys
from tqdm import tqdm
import cv2
import platform
platform.architecture()
sys.path.append("/content/openpilot")
from tools.lib.logreader import LogReader
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tools.lib.framereader import FrameReader
import h5py

position: /content
CEREAL_PATH: /content/openpilot/cereal


In [ ]:
!git clone "https://github.com/commaai/comma2k19.git"

Cloning into 'comma2k19'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 152 (delta 20), reused 15 (delta 15), pack-reused 119 (from 1)
Receiving objects: 100% (152/152), 56.17 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
example_segment = '/content/comma2k19/Example_1/b0c9d2329ad1606b|2018-08-02--08-34-47/40/'
lr = LogReader(example_segment + 'raw_log.bz2')
# make list of logs
logs = list(lr)
[l.which() for l in logs[:50]]

['sensorEventsDEPRECATED',
 'controlsState',
 'sendcan',
 'carState',
 'carControl',
 'logMessage',
 'can',
 'longitudinalPlan',
 'can',
 'sendcan',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'carState',
 'controlsState',
 'carControl',
 'logMessage',
 'logMessage',
 'logMessage',
 'can',
 'roadEncodeIdx',
 'roadEncodeIdx',
 'longitudinalPlan',
 'controlsState',
 'sendcan',
 'carState',
 'carControl',
 'sensorEventsDEPRECATED',
 'can',
 'logMessage',
 'logMessage',
 'radarState',
 'liveTracksDEPRECATED',
 'can',
 'sensorEventsDEPRECATED',
 'liveLongitudinalMpcDEPRECATED',
 'can',
 'longitudinalPlan',
 'liveLongitudinalMpcDEPRECATED',
 'sendcan',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'controlsState',
 'carState',
 'carControl',
 'can',
 'roadCameraState',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED']

In [ ]:
# We can extract frames from the video with framereader
# from openpilot tools, we look at frame 600
frame_index = 600

fr = FrameReader(example_segment + 'video.hevc')
#figsize(12,12)
#imshow(fr.get(frame_index, pix_fmt='rgb24')[0]);
#title('Frame 600 extracted from video with FrameReader', fontsize=25);

In [ ]:
def get_sample(p):
    frame_reader = FrameReader(p+'/video.hevc')
    logs = list(LogReader(p + '/raw_log.bz2'))

    angle = np.array([l.carState.steeringAngleDeg for l in logs if l.which() == 'carState'])[1::5][1::5]
    time = np.array([l.logMonoTime for l in logs if l.which() == 'carState'])[1::5][1::5]
    vEgo = np.array([l.carState.vEgo for l in logs if l.which() == 'carState'])[1::5][1::5]
    gas = np.array([l.carState.gas for l in logs if l.which() == 'carState'])[1::5][1::5]
    brake = np.array([l.carState.brake for l in logs if l.which() == 'carState'])[1::5][1::5]
    gps_times = np.load(p + '/global_pose/frame_gps_times')
    times = np.load(p + '/global_pose/frame_times')
    gas = np.array([l.carState.gas for l in logs if l.which() == 'carState'])[1::5][1::5]
    gaspressed = np.array([l.carState.gasPressed for l in logs if l.which() == 'carState'])[1::5][1::5]
    brake = np.array([l.carState.brake for l in logs if l.which() == 'carState'])[1::5][1::5]
    brake_pressed = np.array([l.carState.brakePressed for l in logs if l.which() == 'carState'])[1::5][1::5]

    enabled = np.array([l.carState.cruiseState.enabled for l in logs if l.which() == 'carState'])[1::5][1::5]
    speed = np.array([l.carState.cruiseState.speed for l in logs if l.which() == 'carState'])[1::5][1::5]
    speedOffset = np.array([l.carState.cruiseState.speedOffset for l in logs if l.which() == 'carState'])[1::5][1::5]
    standstill = np.array([l.carState.cruiseState.standstill for l in logs if l.which() == 'carState'])[1::5][1::5]
    nonAdaptive = np.array([l.carState.cruiseState.nonAdaptive for l in logs if l.which() == 'carState'])[1::5][1::5]
    speedCluster = np.array([l.carState.cruiseState.speedCluster for l in logs if l.which() == 'carState'])[1::5][1::5]

    leftBlinker = np.array([l.carState.leftBlinker for l in logs if l.which() == 'carState'])[1::5][1::5]
    rightBlinker = np.array([l.carState.rightBlinker for l in logs if l.which() == 'carState'])[1::5][1::5]
    #print(frame_reader.frame_count, gps_times.shape, times.shape)
    #print([l.carState for l in logs if l.which() == "carState"][0])
    #print([l.radarState for l in logs if l.which() == "radarState"][0])

    dist = np.array([l.radarState.leadOne.dRel for l in logs if l.which() == "radarState"])[1::5]
    if ((vEgo == 0).mean() > 0.2) or ((dist == 0).mean() > 0.2) or len(dist) <=230:
        return None
    images = []
    l = list(range(frame_reader.frame_count))
    if len(l) > 245:
        l = l[1::5]
    for idx in list(range(frame_reader.frame_count))[1::5]:
        image = np.array(frame_reader.get(idx, pix_fmt='rgb24')[0], dtype=np.float64)
        image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        images.append(image)
    steady_state = ~gaspressed & ~brake_pressed & ~leftBlinker & ~rightBlinker
    last_idx = 0
    desired_gap = np.zeros(steady_state.shape)

    for i in range(len(steady_state)-1):
        if steady_state[i] == True:
            desired_gap[last_idx:i] = int(dist[i])
            last_idx = i

    sample = {
        'image': images,
        "CruiseStateenabled": enabled,
        "CruiseStatespeed": speed,
        "CruiseStatespeedOffset": speedOffset,
        "CruiseStatestandstill": standstill,
        "CruiseStatenonAdaptive": nonAdaptive,
        "CruiseStatespeedCluster": speedCluster,
        'leftBlinker': leftBlinker,
        'rightBlinker': rightBlinker,
        "gas": gas,
        "gaspressed": gaspressed,
        "brake": brake,
        "brakepressed": brake_pressed,
        'angle': angle,
        'time': time,
        'gas': gas,
        'vEgo': vEgo,
        'brake': brake,
        'dist': dist,
        'desired_dist': desired_gap,
        }
    return sample if not ((desired_gap == 0).mean() > 0.2) else None

In [ ]:
def save_h5py(i, sample, h):
    group = h.create_group(str(i))
    for col in sample.keys():
            dt = np.float32 if col != 'image' else int#
            dataset_name = col #groups are divided by '/'
            a = list(sample[col])
            group.create_dataset(dataset_name, data = np.asarray(a, dtype=dt),
                    #compression_opts=9,
                    #chunks=(164, 20, 20, 3),
                    compression='lzf')

In [ ]:

# Vai nella cartella
%cd /content/dataset

# Scarica il primo chunk (~9GB)
!wget https://huggingface.co/datasets/commaai/comma2k19/resolve/main/Chunk_1.zip?download=true


[Errno 2] No such file or directory: '/content/dataset'
/content
--2025-05-21 13:34:24--  https://huggingface.co/datasets/commaai/comma2k19/resolve/main/Chunk_1.zip?download=true
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/07/db/07dbdc4afc2545f4890b9e714253fb2cd7bce5471348d923dd7e55a3e710884f/b7f18b77ca5980e36d536344995bcfaffd4cbe6b84fe43ca443ea20c26547e30?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Chunk_1.zip%3B+filename%3D%22Chunk_1.zip%22%3B&response-content-type=application%2Fzip&Expires=1747838064&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzgzODA2NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8wNy9kYi8wN2RiZGM0YWZjMjU0NWY0ODkwYjllNzE0MjUzZmIyY2Q3YmNlNTQ3MTM0OGQ5MjNkZDdlNTVhM2U3MTA4ODRmL2I3

In [ ]:
!mkdir -p /content/dataset
!mv "Chunk_1.zip?download=true" Chunk_1.zip
!mv Chunk_1.zip /content/dataset

In [ ]:
!wget https://huggingface.co/datasets/commaai/comma2k19/resolve/main/Chunk_2.zip?download=true

In [ ]:
!wget https://huggingface.co/datasets/commaai/comma2k19/resolve/main/Chunk_3.zip?download=true

In [ ]:
!unzip /content/dataset/Chunk_1.zip -d /content/dataset

In [ ]:
dataset_type = "train"
main_dir='/content/dataset/Chunk_1/'
hdf5_filename = "gas_and_brake_train_comma_chunk_1_w_imgs.hdf5"
h_train = h5py.File(hdf5_filename, 'w')
hdf5_filename = "gas_and_brake_val_comma_chunk_1_w_imgs.hdf5"
h_val = h5py.File(hdf5_filename, 'w')
hdf5_filename = "gas_and_brake_test_comma_chunk_1_w_imgs.hdf5"
h_test = h5py.File(hdf5_filename, 'w')

In [ ]:
for j, drive_sequence_path in tqdm(enumerate(os.listdir(main_dir))):
    if '.DS_Store ' in drive_sequence_path or not os.path.isdir(main_dir+"/"+drive_sequence_path): continue
    min_sequence_paths = os.listdir(main_dir+"/"+drive_sequence_path)
    if len(min_sequence_paths) < 3: continue
    min_sequence_path_test = main_dir+"/"+drive_sequence_path+"/"+min_sequence_paths[-2]
    min_sequence_paths_val = main_dir+"/"+drive_sequence_path+"/"+min_sequence_paths[-1]
    sample = get_sample(min_sequence_path_test)
    if sample != None:
        save_h5py(f"{drive_sequence_path}", sample, h_test)
    sample = get_sample(min_sequence_paths_val)

    if sample != None:
        save_h5py(f"{drive_sequence_path}", sample, h_val)
    for i, min_sequence in enumerate(min_sequence_paths[:-2]):
        if '.DS_Store ' in min_sequence or not os.path.isdir(main_dir+"/"+drive_sequence_path+'/'+min_sequence): continue
        p = main_dir+"/"+drive_sequence_path+"/"+min_sequence
        sample = get_sample(p)
        if sample != None:
            save_h5py(f"{drive_sequence_path}_{min_sequence}", sample, h_train)


22it [1:42:27, 279.45s/it]


In [ ]:
h_test.close()
h_train.close()
h_val.close()

In [ ]:
from google.colab import files
files.download("/content/gas_and_brake_train_comma_chunk_1_w_imgs.hdf5")
files.download("/content/gas_and_brake_val_comma_chunk_1_w_imgs.hdf5")
files.download("/content/gas_and_brake_test_comma_chunk_1_w_imgs.hdf5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Clean Up Data

In [ ]:
import h5py
import numpy as np
from tqdm import tqdm
from scipy import ndimage
import os

# 📁 Directory dove si trovano i file chunk_1
base_path = "/content"  # cambia se lavori in locale
dataset_types = ['train', 'val', 'test']

for dataset_type in dataset_types:
    input_path = os.path.join(base_path, f"gas_and_brake_{dataset_type}_comma_chunk_1_w_imgs.hfd5")
    output_path = os.path.join(base_path, f"filtered_chunk1_{dataset_type}.hdf5")

    print(f"🔄 Processing {dataset_type} from: {input_path}")

    if not os.path.exists(input_path):
        print(f"❌ File not found: {input_path}")
        continue

    with h5py.File(input_path, "r") as h5_file, h5py.File(output_path, "w") as h_out:
        keys = list(h5_file.keys())

        # Opzionale: escludi chiavi problematiche
        # bad_keys = ['37', '53', '55', '58']
        # keys = [k for k in keys if k not in bad_keys]

        for key in tqdm(keys, desc=f"Processing {dataset_type}"):
            group_in = h5_file[key]

            if 'desired_dist' not in group_in:
                continue

            desired = np.array(group_in['desired_dist'][()])
            filtered = ndimage.median_filter(desired, size=12)

            if (filtered == 0).mean() > 0.2:
                continue  # scarta campioni poco informativi

            group_out = h_out.create_group(key)

            for col in group_in.keys():
                dt = np.float32 if col != 'image' else int
                group_out.create_dataset(
                    col,
                    data=group_in[col],
                    compression='gzip',
                    compression_opts=6,
                    chunks=True
                )

    print(f"✅ Salvato: {output_path}\n")

In [1]:
import numpy as np
import h5py
from scipy import ndimage
from tqdm import
base_path = "/content"  # cambia se lavori in locale
dataset_types = ['train', 'val', 'test']

In [ ]:
if dataset_type == "train":
            data_path = "/content/gas_and_brake_train_comma_chunk_1_w_imgs.hfd5"
elif dataset_type == "test":
            data_path = "/content/gas_and_brake_test_comma_chunk_1_w_imgs.hfd5"
elif dataset_type == "val":
            data_path = "/content/gas_and_brake_val_comma_chunk_1_w_imgs.hfd5"
people_seqs = []
h5_file = h5py.File(data_path, "r")
keys = list(h5_file.keys())
if dataset_type == "train":
            h5_file2 = h5py.File(data_path2, "r")
            keys2 = list(h5_file2.keys())
            keys = list(np.array(keys))
            keys2 = list(np.array(keys2))
            h5_file3 = h5py.File(data_path3, "r")
            keys3 = list(h5_file3.keys())

if dataset_type == "val":
            keys = np.array(keys)
            h5_file2 = h5py.File(data_path2, "r")
            keys2 = list(h5_file2.keys())
            h5_file3 = h5py.File(data_path3, "r")
            keys3 = list(h5_file3.keys())

if dataset_type == "test":
            keys = np.array(keys)
            h5_file2 = h5py.File(data_path2, "r")
            keys2 = list(h5_file2.keys())
            h5_file3 = h5py.File(data_path3, "r")
            keys3 = list(h5_file3.keys())

In [ ]:
h = h5py.File(f"/data1/jessica/data/toyota/comma_{dataset_type}_w_desired_filtered.h5py", 'w')

In [ ]:
for key in tqdm(keys):
    data = h5_file[key]
    group  = h.create_group(key)
    for col in data.keys():
        dt = np.float32 if col != 'image' else int#
        dataset_name = col #groups are divided by '/'
        a = data[col]
        group.create_dataset(dataset_name, data = a, compression_opts=6, compression='gzip', chunks=True)



In [ ]:
for key in tqdm(h.keys()):
    data = h[key]
    t = np.array((data['desired_dist'][()]))
    result = ndimage.median_filter(t, size=12)
    print((result == 0).mean())
    if not (len(data.keys()) >= 5):
        print(key, data.keys())

In [ ]:
data = h5_file4['99c94dc769b5d96e|2018-05-13--20-53-38_29']
data_new = h['99c94dc769b5d96e|2018-05-13--20-53-38_29']

In [ ]:
for col in data.keys():
    if col in ['angle', 'brake', 'dist', 'gas']: continue
    print(key,col)
    dt = np.float32 if col != 'image' else int#
    dataset_name = col #groups are divided by '/'
    a = data[col]
    data_new.create_dataset(dataset_name, data = a, compression_opts=6, compression='gzip')

In [ ]:
data_new = h['99c94dc769b5d96e|2018-05-13--20-53-38_29']
data_new

In [ ]:
for key in tqdm(keys4):
    data = h5_file4[key]
    if key != '99c94dc769b5d96e|2018-05-13--20-53-38_29':
        continue
    group  = h.create_group(key)
    for col in data.keys():
        if col in ['angle', 'brake', 'dist', 'gas']: continue
        print(key,col)
        dt = np.float32 if col != 'image' else int#
        dataset_name = col #groups are divided by '/'
        a = data[col]
        group.create_dataset(dataset_name, data = a, compression_opts=6, compression='gzip')

In [ ]:
h.close()

In [ ]:

!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-venv python3.10-dev -y

!python3.10 -m venv py310env

!py310env/bin/pip install --upgrade pip
!py310env/bin/pip install git+https://github.com/openai/CLIP.git

!py310env/bin/pip install \
    h5py==3.8.0 \
    humanize==4.6.0 \
    local_attention==1.8.6 \
    matplotlib==3.7.1 \
    numba==0.56.4 \
    numpy==1.23.5 \
    opencv_python==4.7.0.72 \
    pandas==1.5.3 \
    Pillow==9.5.0 \
    psutil==5.9.4 \
    pytorch_lightning==1.9.4 \
    PyYAML==6.0 \
    scikit_learn==1.2.2 \
    scipy==1.10.1 \
    seaborn==0.12.2 \
    timm==0.9.1 \
    torch==2.0.0 \
    torchvision==0.15.1 \
    tqdm==4.65.0 \
    transformers==4.29.1
!py310env/bin/pip install tensorboard

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,944 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 Packages [77.3 kB]
Get:13 http://arch

In [ ]:
!git clone https://github.com/Gabriele-Raffaele/concept_gridlock.git
!git clone https://github.com/commaai/comma2k19.git



fatal: destination path 'concept_gridlock' already exists and is not an empty directory.
fatal: destination path 'comma2k19' already exists and is not an empty directory.


In [ ]:
%cd concept_gridlock

[Errno 2] No such file or directory: 'concept_gridlock'
/content/concept_gridlock/comma_preprocess


/bin/bash: line 1: ../py310env/bin/python: No such file or directory


In [ ]:
!../py310env/bin/python main.py -task distance -train -gpu_num 0 -dataset comma -backbone clip -bs 4 -ground_truth normal

100%|███████████████████████████████████████| 338M/338M [00:24<00:00, 14.6MiB/s]
using clip backbone
/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_clip_None_1
⚠️ Nessuna versione precedente trovata, si procede senza resume.
RESUME FROM: None
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type    | Params
-------------------------------------
0 | model    | VTN     | 167 M 
1 | bce_loss | BCELoss | 0     
-------------------------------------
167 M     Trainable params
0         Non-trainable params
167 M     Total params
668.211   Total estimated model params size (MB)
Sanity Checking: 0it [00:00, ?it/s]Traceback (most recent call last):
  File "/content/concept_gridlock/main.py", line 116, in <module>
    trainer.fit(module)
  File "/content/py310env/lib/python3.10/site-packages/pytorch_lightning/trainer/